## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..
import os, sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))

/Users/Tony/Other Docs/distilling-and-forgetting-in-large-pre-trained-models


In [3]:
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
from datasets import load_from_disk

import matplotlib.pyplot as plt
import seaborn as sns

from evaluation.string_edit_metrics import get_string_edit_metrics, get_string_edit_metrics_ortho_and_norm
from normalization.whisper_normalization import get_whisper_normalizer
from utils.whisper_hallucinations.eval_filter_criterion import eval_filter_criterion
from utils.whisper_hallucinations.get_features import compute_gzip_compression_ratio
from utils.notebook_utils import listen_to_audio

whisper_norm = get_whisper_normalizer("english")
sns.set_theme(context="paper", style="ticks")

## Load cached dataset

In [4]:
dataset_name = "ami_validation"

savepath = f"notebooks/data/whisper_hallucinations_cached_ds/{dataset_name}"
ds = load_from_disk(savepath)

ds.features

{'text': Value(dtype='string', id=None),
 'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'teacher_text': Value(dtype='string', id=None),
 'teacher_labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'audio_length': Value(dtype='float64', id=None),
 'n_tokens_labels': Value(dtype='int64', id=None),
 'n_tokens_teacher': Value(dtype='int64', id=None),
 'diff_n_tokens': Value(dtype='int64', id=None),
 'gzip_ratio': Value(dtype='float64', id=None),
 'teacher_gzip_ratio': Value(dtype='float64', id=None),
 'diff_gzip_ratio': Value(dtype='float64', id=None),
 'n_overlaps': Value(dtype='int64', id=None)}

## Convert to DataFrame

In [5]:
df = pd.DataFrame({col: ds[col] for col in ds.features.keys() if col != "audio"})

df.head()

,text,labels,teacher_text,teacher_labels,audio_length,n_tokens_labels,n_tokens_teacher,diff_n_tokens,gzip_ratio,teacher_gzip_ratio,diff_gzip_ratio,n_overlaps
0,but like mobile phones have screens and they'r...,"[50258, 50363, 5955, 411, 6013, 10216, 362, 11...",but like mobile phones have screens and they'...,"[50258, 50363, 457, 411, 6013, 10216, 362, 111...",2.68,14,15,1,0.757143,0.774648,0.017505,0
1,mm,"[50258, 50363, 2174, 50257]",you,"[50258, 50363, 291, 50257]",0.15,4,4,0,0.090909,0.166667,0.075758,0
2,furry,"[50258, 50363, 69, 30614, 50257]",furry.,"[50258, 50363, 47073, 13, 50257]",0.52,5,5,0,0.200000,0.259259,0.059259,0
3,yeah,"[50258, 50363, 19650, 50257]",yeah.,"[50258, 50363, 1338, 13, 50257]",1.86,4,5,1,0.166667,0.230769,0.064103,0
4,i mean it just seems like yeah,"[50258, 50363, 72, 914, 309, 445, 2544, 411, 1...",i mean it just seems like... yeah.,"[50258, 50363, 741, 914, 309, 445, 2544, 411, ...",2.27,10,12,2,0.600000,0.636364,0.036364,0


## First analysis

In [ ]:
dict_string_edit_metrics = get_string_edit_metrics_ortho_and_norm(references=df["text"], predictions=df["teacher_text"], norm_fn=get_whisper_normalizer("english"))

dict_string_edit_metrics

**Observation:** Using the Whisper normalizer drastically decreased the different string edit metric errors. Since 1-best operated without any normalization, we will focus on the orthographic WER in this study.

In [ ]:
plt.figure(figsize=(5, 3))
df["audio_length"].plot.hist();

In [ ]:
plt.figure(figsize=(12, 3))
sns.boxplot(data=df[["n_tokens_labels", "n_tokens_teacher"]], orient="h");

In [ ]:
x_col = "n_tokens_labels"
y_col = "n_tokens_teacher"

sns.jointplot(data=df, x=x_col, y=y_col, alpha=0.5)
line_max_coord = min(df[x_col].max(), df[y_col].max())
plt.plot([0, line_max_coord], [0, line_max_coord], 'b--', label=r"$y=x$")
plt.legend()
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 1.5))
sns.boxplot(data=df[["diff_n_tokens"]], orient="h");

In [ ]:
plt.figure(figsize=(12, 1.5))
sns.boxplot(data=df[["diff_n_tokens"]], orient="h")
plt.xlim(-50, 50);

In [ ]:
sns.scatterplot(data=df, x="audio_length", y="diff_n_tokens");

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(8, 3), sharey=True)
sns.scatterplot(data=df, x="audio_length", y="n_tokens_labels", label="Labels", alpha=0.2, ax=axis[0])
sns.scatterplot(data=df, x="audio_length", y="n_tokens_teacher", label="Predictions", c="coral", alpha=0.2, ax=axis[1])
axis[0].set_ylabel("n_tokens")
fig.tight_layout()

## Criteria for filtering

### Difference in number of tokens

In [ ]:
THRESH_DELTA_N_TOKENS = int(df["diff_n_tokens"].mean() + 1.5 * df["diff_n_tokens"].std())
THRESH_DELTA_N_TOKENS

In [ ]:
# Define the columns to use for the plot
x_col = "n_tokens_labels"
y_col = "n_tokens_teacher"

# Define the value of delta
delta = THRESH_DELTA_N_TOKENS

# Create the joint plot
sns.jointplot(data=df, x=x_col, y=y_col, alpha=0.5)

# Plot the line y = x + delta
plt.plot([df[x_col].min(), df[x_col].max()], [df[x_col].min() + delta, df[x_col].max() + delta], 'r--', label=r"$y = x + \delta$")

# Fill the area above the line y = x + delta
plt.fill_between([0, df[x_col].max()],
                 [0 + delta, df[x_col].max() + delta], df[y_col].max(),
                 alpha=0.2,
                 color="red")

# Plot the line y = x
plt.plot([df[x_col].min(), df[x_col].max()], [df[x_col].min(), df[x_col].max()], 'k--', label=r"$y = x$")

# Add legend and set axis limits
plt.legend()
plt.xlim(0, df[x_col].max())
plt.ylim(0, df[y_col].max())

# Adjust the layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Define the columns to use for the plot
x_col = "n_tokens_labels"
y_col = "n_tokens_teacher"

# Define the value of delta
delta = THRESH_DELTA_N_TOKENS

# Create the joint plot
sns.jointplot(data=df, x=x_col, y=y_col, alpha=0.5)

# Plot the line y = x + delta
plt.plot([df[x_col].min(), df[x_col].max()], [df[x_col].min() + delta, df[x_col].max() + delta], 'r--', label=r"$y = x + \delta$")

# Fill the area above the line y = x + delta
plt.fill_between([0, df[x_col].max()],
                 [0 + delta, df[x_col].max() + delta], df[y_col].max(),
                 alpha=0.2,
                 color="red")

# Plot the line y = x
plt.plot([df[x_col].min(), df[x_col].max()], [df[x_col].min(), df[x_col].max()], 'k--', label=r"$y = x$")

# Add legend and set axis limits
plt.legend()
line_max_coord = min(df[x_col].max(), df[y_col].max())
plt.xlim(0, line_max_coord)
plt.ylim(0, line_max_coord)

# Adjust the layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))

# create a histogram of the "diff_n_tokens" column
sns.histplot(data=df, x="diff_n_tokens", bins=100)

# shade the area that will be filtered out in red
plt.axvspan(THRESH_DELTA_N_TOKENS, df["diff_n_tokens"].max(), color='red', alpha=0.2, label="Filtered out")

plt.legend()
plt.xlim(-20, 50)

# show the plot
plt.tight_layout()
plt.show()

In [ ]:
df["high_diff_n_tokens"] = (df["diff_n_tokens"] > THRESH_DELTA_N_TOKENS)

df["high_diff_n_tokens"].value_counts()

In [ ]:
df_candidates = df[df["high_diff_n_tokens"]]

for idx in range(len(df_candidates)):
    print(f"Idx = {df_candidates.iloc[idx].name}")
    print("Reference: ", df_candidates.iloc[idx]["text"])
    print("Prediction: ", df_candidates.iloc[idx]["teacher_text"])
    print()

In [ ]:
list_idx_to_listen = df_candidates.index[:5]

listen_to_audio(ds, list_idx_to_listen, pred_col="teacher_text")

In [ ]:
eval_filter_criterion(ds, filter_fn=lambda x: x["diff_n_tokens"] <= THRESH_DELTA_N_TOKENS)

**Observation:** Good criterion. Note that this also tackles the fact that the vanilla Whisper models tend to transcribe even the speeches in the background. By removing them from the training set, we hope to teach Whisper to only focus on the main speaker.

### gzip compression ratio

In [ ]:
# Quick example to confirm the intuition:
ref = "yeah so uh what we'll do is uh"
pred = "so, what we will do is, we will do is, we will do is, we will do is, we will do is,"

print("gzip ratios:")
print("- reference: ", compute_gzip_compression_ratio(ref))
print("- prediction: ", compute_gzip_compression_ratio(pred))

In [ ]:
df.columns

In [ ]:
x_col = "gzip_ratio"
y_col = "teacher_gzip_ratio"

sns.jointplot(data=df, x=x_col, y=y_col, alpha=0.5)
line_max_coord = min(df[x_col].max(), df[y_col].max())
plt.plot([0, line_max_coord], [0, line_max_coord], 'b--', label=r"$y=x$")
plt.legend()
# plt.xlim(-1, line_max_coord)
# plt.ylim(-1, line_max_coord)
plt.tight_layout()

In [ ]:
x_col = "gzip_ratio"
y_col = "teacher_gzip_ratio"

sns.jointplot(data=df, x=x_col, y=y_col, alpha=0.5)
line_max_coord = min(df[x_col].max(), df[y_col].max())
plt.plot([0, line_max_coord], [0, line_max_coord], 'b--', label=r"$y=x$")
plt.legend()
plt.xlim(0, line_max_coord)
plt.ylim(0, 2.5)
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 1.5))
sns.boxplot(data=df[["diff_gzip_ratio"]], orient="h");

In [ ]:
THRESH_DIFF_GZIP = df["diff_gzip_ratio"].mean() + 1.5 * df["diff_gzip_ratio"].std()

THRESH_DIFF_GZIP

In [ ]:
# Define the columns to use for the plot
x_col = "gzip_ratio"
y_col = "teacher_gzip_ratio"


# Define the value of delta
delta = THRESH_DIFF_GZIP

# Create the joint plot
sns.jointplot(data=df, x=x_col, y=y_col, alpha=0.5)

# Plot the line y = x + delta
plt.plot([df[x_col].min(), df[x_col].max()], [df[x_col].min() + delta, df[x_col].max() + delta], 'r--', label=r"$y = x + \delta$")

# Fill the area above the line y = x + delta
plt.fill_between([0, df[x_col].max()],
                 [0 + delta, df[x_col].max() + delta], df[y_col].max(),
                 alpha=0.2,
                 color="red")

# Plot the line y = x
plt.plot([df[x_col].min(), df[x_col].max()], [df[x_col].min(), df[x_col].max()], 'k--', label=r"$y = x$")

# Plot the line y = x - delta
plt.plot([df[x_col].min(), df[x_col].max()], [df[x_col].min() - delta, df[x_col].max() - delta], 'r--', label=r"$y = x - \delta$")

# Fill the area below the line y = x - delta
plt.fill_between([0, df[x_col].max()],
                 [0 - delta, df[x_col].max() - delta], df[y_col].min(),
                 alpha=0.2,
                 color="red")

# Add legend and set axis limits
plt.legend()
plt.xlim(0, df[x_col].max())
plt.ylim(0, df[y_col].max())

# Adjust the layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Define the columns to use for the plot
x_col = "gzip_ratio"
y_col = "teacher_gzip_ratio"


# Define the value of delta
delta = THRESH_DIFF_GZIP

# Create the joint plot
sns.jointplot(data=df, x=x_col, y=y_col, alpha=0.5)

# Plot the line y = x + delta
plt.plot([df[x_col].min(), df[x_col].max()], [df[x_col].min() + delta, df[x_col].max() + delta], 'r--', label=r"$y = x + \delta$")

# Fill the area above the line y = x + delta
plt.fill_between([0, df[x_col].max()],
                 [0 + delta, df[x_col].max() + delta], df[y_col].max(),
                 alpha=0.2,
                 color="red")

# Plot the line y = x
plt.plot([df[x_col].min(), df[x_col].max()], [df[x_col].min(), df[x_col].max()], 'k--', label=r"$y = x$")

# Plot the line y = x - delta
plt.plot([df[x_col].min(), df[x_col].max()], [df[x_col].min() - delta, df[x_col].max() - delta], 'r--', label=r"$y = x - \delta$")

# Fill the area below the line y = x - delta
plt.fill_between([0, df[x_col].max()],
                 [0 - delta, df[x_col].max() - delta], df[y_col].min(),
                 alpha=0.2,
                 color="red")

# Add legend and set axis limits
plt.legend()
line_max_coord = min(df[x_col].max(), df[y_col].max())
plt.xlim(0, line_max_coord)
plt.ylim(0, line_max_coord)

# Adjust the layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))

# create a histogram of the "diff_n_tokens" column
sns.histplot(data=df, x="diff_gzip_ratio", bins=200)

# shade the area that will be filtered out in red
plt.axvspan(THRESH_DIFF_GZIP, df["diff_gzip_ratio"].max(), color='red', alpha=0.2, label="Filtered out")
plt.axvspan(-THRESH_DIFF_GZIP, df["diff_gzip_ratio"].min(), color='red', alpha=0.2)

plt.legend()
plt.xlim(-1, 1)

# show the plot
plt.tight_layout()
plt.show()

In [ ]:
df_candidates = df[df["diff_gzip_ratio"] > THRESH_DIFF_GZIP]

for idx in range(len(df_candidates)):
    print(f"Idx = {df_candidates.iloc[idx].name}")
    print("Reference: ", df_candidates.iloc[idx]["text"])
    print("Prediction: ", df_candidates.iloc[idx]["teacher_text"])
    print()

In [ ]:
df_candidates = df[df["diff_gzip_ratio"] < - THRESH_DIFF_GZIP]

for idx in range(len(df_candidates)):
    print(f"Idx = {df_candidates.iloc[idx].name}")
    print("Reference: ", df_candidates.iloc[idx]["text"])
    print("Prediction: ", df_candidates.iloc[idx]["teacher_text"])
    print()

In [ ]:
eval_filter_criterion(ds, filter_fn=lambda x: np.abs(x["diff_gzip_ratio"]) <= THRESH_DIFF_GZIP)

### Timestamp overlaps

In [ ]:
df["n_overlaps"].value_counts()

In [ ]:
ds_candidate = ds.filter(lambda x: x["n_overlaps"] > 0)

for idx, row in enumerate(ds_candidate):
    print("Index: ", idx)
    print("Reference: ", row["text"])
    print("Pred: ", row["teacher_text"])
    print()

In [ ]:
eval_filter_criterion(ds, filter_fn=lambda x: x["n_overlaps"] == 0)